In [ ]:
from setuptools import setup, find_packages

setup(

    name='llm_agents',

    version='0.1.0',

    description='A package for building agents which use the OpenAI API to figure out actions to take and can use tools.',

    author='Marc Päpper',

    author_email='marc@paepper.com',

    url='https://github.com/mpaepper/llm_agents',

    packages=find_packages(),

    install_requires=[

        'google-search-results>=2.4.2',

        'openai>=0.27.0',

        'pydantic>=1.10.5',

        'requests>=2.28.2'

    ],

    classifiers=[

        'Programming Language :: Python :: 3',

        'License :: OSI Approved :: MIT License'

    ],

)

In [ ]:
from llm_agents.agent import Agent

from llm_agents.llm import ChatLLM

from llm_agents.tools.python_repl import PythonREPLTool

from llm_agents.tools.hackernews import HackerNewsSearchTool

from llm_agents.tools.search import SerpAPITool

from llm_agents.tools.searx import SearxSearchTool

from llm_agents.tools.google_search import GoogleSearchTool

__all__ = ['Agent', 'ChatLLM', 'PythonREPLTool',

           'HackerNewsSearchTool', 'SerpAPITool', 'SearxSearchTool', 'GoogleSearchTool']

In [ ]:
import openai

import os

from pydantic import BaseModel

from typing import List

class ChatLLM(BaseModel):

    model: str = 'gpt-3.5-turbo'

    temperature: float = 0.0

    openai.api_key = os.environ["OPENAI_API_KEY"]  # Credentials setup

    def generate(self, prompt: str, stop: List[str] = None):

        response = openai.ChatCompletion.create(

            model=self.model,

            messages=[{"role": "user", "content": prompt}],

            temperature=self.temperature,

            stop=stop

        )

        return response.choices[0].message.content

if __name__ == '__main__':

    llm = ChatLLM()

    result = llm.generate(prompt='Who is the president of the USA?')

    print(result)

In [ ]:
import sys

from io import StringIO

from typing import Dict, Optional

from pydantic import BaseModel, Field

from llm_agents.tools.base import ToolInterface

# Taken from https://github.com/hwchase17/langchain/blob/master/langchain/python.py

class PythonREPL(BaseModel):

    """Simulates a standalone Python REPL."""

    globals: Optional[Dict] = Field(default_factory=dict, alias="_globals")

    locals: Optional[Dict] = Field(default_factory=dict, alias="_locals")

    def run(self, command: str) -> str:

        """Run command with own globals/locals and returns anything printed."""

        old_stdout = sys.stdout
                sys.stdout = mystdout = StringIO()

        try:

            exec(command, self.globals, self.locals)

            sys.stdout = old_stdout

            output = mystdout.getvalue()

        except Exception as e:

            sys.stdout = old_stdout

            output = str(e)

        return output

def _get_default_python_repl() -> PythonREPL:

    return PythonREPL(_globals=globals(), _locals=None)

class PythonREPLTool(ToolInterface):

    """A tool for running python code in a REPL."""

    name: str = "Python REPL"

    description: str = (

        "A Python shell. Use this to execute python commands. "

        "Input should be a valid python command. "

        "If you want to see the output of a value, you should print it out "

        "with `print(...)`."

    )

    python_repl: PythonREPL = Field(default_factory=_get_default_python_repl)

    def use(self, input_text: str) -> str:

        input_text = input_text.strip().strip("```")

        return self.python_repl.run(input_text)

if __name__ == '__main__':

    repl_tool = PythonREPLTool()

    result = repl_tool.use('print(5 * 7)')

    assert result == "35\n"

    print(result)

In [ ]:
import datetime

import re

from pydantic import BaseModel

from typing import List, Dict, Tuple

from llm_agents.llm import ChatLLM

from llm_agents.tools.base import ToolInterface

from llm_agents.tools.python_repl import PythonREPLTool

FINAL_ANSWER_TOKEN = "Final Answer:"

OBSERVATION_TOKEN = "Observation:"

THOUGHT_TOKEN = "Thought:"

PROMPT_TEMPLATE = """Today is {today} and you can use tools to get new information. Answer the question as best as you can using the following tools:

{tool_description}

Use the following format:

Question: the input question you must answer

Thought: comment on what you want to do next

Action: the action to take, exactly one element of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)

Thought: I now know the final answer

Final Answer: your final answer to the original input question

Begin!

Question: {question}

Thought: {previous_responses}

"""

class Agent(BaseModel):

    llm: ChatLLM

    tools: List[ToolInterface]

    prompt_template: str = PROMPT_TEMPLATE

    max_loops: int = 15

    # The stop pattern is used, so the LLM does not hallucinate until the end

    stop_pattern: List[str] = [f'\n{OBSERVATION_TOKEN}', f'\n\t{OBSERVATION_TOKEN}']

    @property

    def tool_description(self) -> str:

        return "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

    @property

    def tool_names(self) -> str:

        return ",".join([tool.name for tool in self.tools])

    @property

    def tool_by_names(self) -> Dict[str, ToolInterface]:

        return {tool.name: tool for tool in self.tools}

    def run(self, question: str):

        previous_responses = []

        num_loops = 0

        prompt = self.prompt_template.format(

                today = datetime.date.today(),

                tool_description=self.tool_description,

                tool_names=self.tool_names,

                question=question,

                previous_responses='{previous_responses}'

        )

        print(prompt.format(previous_responses=''))

        while num_loops < self.max_loops:

            num_loops += 1

            curr_prompt = prompt.format(previous_responses='\n'.join(previous_responses))

            generated, tool, tool_input = self.decide_next_action(curr_prompt)

            if tool == 'Final Answer':

                return tool_input

            if tool not in self.tool_by_names:

                raise ValueError(f"Unknown tool: {tool}")

            tool_result = self.tool_by_names[tool].use(tool_input)

            generated += f"\n{OBSERVATION_TOKEN} {tool_result}\n{THOUGHT_TOKEN}"

            print(generated)

            previous_responses.append(generated)

    def decide_next_action(self, prompt: str) -> str:

        generated = self.llm.generate(prompt, stop=self.stop_pattern)

        tool, tool_input = self._parse(generated)

        return generated, tool, tool_input

    def _parse(self, generated: str) -> Tuple[str, str]:

        if FINAL_ANSWER_TOKEN in generated:

            return "Final Answer", generated.split(FINAL_ANSWER_TOKEN)[-1].strip()

        regex = r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)"

        match = re.search(regex, generated, re.DOTALL)

        if not match:

            raise ValueError(f"Output of LLM is not parsable for next tool use: `{generated}`")

        tool = match.group(1).strip()

        tool_input = match.group(2)

        return tool, tool_input.strip(" ").strip('"')

if __name__ == '__main__':

    agent = Agent(llm=ChatLLM(), tools=[PythonREPLTool()])

    result = agent.run("What is 7 * 9 - 34 in Python?")

    print(f"Final answer is {result}")

In [ ]:
from llm_agents import Agent, ChatLLM, PythonREPLTool, HackerNewsSearchTool, SerpAPITool

if __name__ == '__main__':

    prompt = input("Enter a question / task for the agent: ")

    agent = Agent(llm=ChatLLM(), tools=[PythonREPLTool(), SerpAPITool(), HackerNewsSearchTool()])

    result = agent.run(prompt)

    print(f"Final answer is {result}")